<a href="https://colab.research.google.com/github/smitike/REU-AI-Cybersecurity-SM/blob/main/REU_AI_Cybersecurity_SM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch
!pip install pennylane
!pip install qiskit
!pip install torchvision
!pip install torchaudio
!pip install torch_geometric
!apt-get update
!apt-get install -y argus-server argus-client
!pip install chardet


  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
ERROR: Operation cancelled by user
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [1]:
!pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 9.7 MB/s eta 0:00:00


# Data Preparation

Use the argus tool to convert the .biagrus file into a readable format (e.g., CSV or directly into a Pandas DataFrame).

Extract network flows and logs from the parsed data.

In [2]:
# Mount Google Drive for file access
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import json

# Load the JSON data and inspect its structure
json_file = '/content/drive/MyDrive/datasets/iot23/iot23.json'  # Change to your actual file path

with open(json_file, 'r') as file:
    data = json.load(file)

# Print the first few levels of the JSON structure to understand where the actual data is
def print_json_structure(data, indent=0):
    if isinstance(data, dict):
        for key, value in data.items():
            print('  ' * indent + str(key))
            if isinstance(value, (dict, list)):
                print_json_structure(value, indent + 1)
    elif isinstance(data, list):
        for i, item in enumerate(data):
            print('  ' * indent + f"[{i}]")
            print_json_structure(item, indent + 1)
    else:
        print('  ' * indent + str(data))

print_json_structure(data)


@context
  @language
  @vocab
  citeAs
  column
  conformsTo
  cr
  data
    @id
    @type
  dataBiases
  dataCollection
  dataType
    @id
    @type
  dct
  extract
  field
  fileProperty
  fileObject
  fileSet
  format
  includes
  isEnumeration
  isLiveDataset
  jsonPath
  key
  md5
  parentField
  path
  personalSensitiveInformation
  recordSet
  references
  regex
  repeated
  replace
  sc
  separator
  source
  subField
  transform
  wd
alternateName
citeAs
conformsTo
license
  @type
  name
  url
distribution
  [0]
    contentUrl
    contentSize
    md5
    encodingFormat
    @id
    @type
    name
    description
  [1]
    contentUrl
    containedIn
      @id
    encodingFormat
    @id
    @type
    name
    description
  [2]
    contentUrl
    containedIn
      @id
    encodingFormat
    @id
    @type
    name
  [3]
    contentUrl
    containedIn
      @id
    encodingFormat
    @id
    @type
    name
  [4]
    contentUrl
    containedIn
      @id
    encodingFormat
    @id
   

In [5]:
import os
import json
import pandas as pd

# Load JSON data
def load_json_data(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data

# Preprocess the JSON data
def preprocess_iot23_data(file_path):
    data = load_json_data(file_path)

    # Extract the relevant data section from the JSON
    records = []
    for record in data.get('recordSet', []):
        record_data = {}
        for field in record.get('field', []):
            column = field.get('source', {}).get('extract', {}).get('column')
            if column:
                record_data[column] = field.get('description')  # Assume description contains the actual data values
        records.append(record_data)

    # Convert the list of records to a DataFrame
    df = pd.DataFrame(records)

    # Inspect the DataFrame to understand its structure
    print("Columns in the DataFrame:", df.columns)

    # Identify and rename the relevant columns
    if 'id.orig_h' in df.columns and 'id.resp_h' in df.columns:
        df.rename(columns={'id.orig_h': 'srcid', 'id.resp_h': 'dstid'}, inplace=True)
    else:
        raise KeyError("Expected columns 'id.orig_h' and 'id.resp_h' not found in the dataset.")

    # Combine source and destination IPs and find unique nodes
    unique_ips = pd.concat([df['srcid'], df['dstid']]).unique()
    num_nodes = len(unique_ips)

    return num_nodes

# Path to your JSON file
json_file = '/content/drive/MyDrive/datasets/iot23/iot23.json'  # Change this to your actual file path

# Load and preprocess the data to find the number of nodes
num_nodes = preprocess_iot23_data(json_file)
print(f"Number of nodes in the dataset: {num_nodes}")


Columns in the DataFrame: Index(['ts', 'uid', 'id.orig_h', 'id.orig_p', 'id.resp_h', 'id.resp_p',
       'proto', 'service', 'duration', 'orig_bytes', 'resp_bytes',
       'conn_state', 'local_orig', 'local_resp', 'missed_bytes', 'history',
       'orig_pkts', 'orig_ip_bytes', 'resp_pkts', 'resp_ip_bytes',
       'tunnel_parents', 'label', 'detailed-label'],
      dtype='object')
Number of nodes in the dataset: 3


In [4]:
import os
import enum
import json
import torch
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from torch_geometric.data import Data
import networkx as nx
import matplotlib.pyplot as plt
import pickle

# Supported datasets
class DatasetType(enum.Enum):
    IOT23 = 0

# Networkx is not precisely made with drawing as its main feature but I experimented with it a bit
class GraphVisualizationTool(enum.Enum):
    NETWORKX = 0,
    IGRAPH = 1

# Data paths
DATA_DIR_PATH = '/content/drive/MyDrive/datasets/iot23'

# All data is stored as pickle
def pickle_read(path):
    with open(path, 'rb') as file:
        data = pickle.load(file)
    return data

def pickle_save(path, data):
    with open(path, 'wb') as file:
        pickle.dump(data, file, protocol=pickle.HIGHEST_PROTOCOL)

def normalize_features(df):
    scaler = StandardScaler()
    scaled_features = scaler.fit_transform(df.values)
    return pd.DataFrame(scaled_features, index=df.index, columns=df.columns)

def build_edge_index(df, src_col, dst_col, add_self_edges=True):
    edges = []
    for _, row in df.iterrows():
        edges.append([row[src_col], row[dst_col]])
    if add_self_edges:
        nodes = set(df[src_col]).union(set(df[dst_col]))
        for node in nodes:
            edges.append([node, node])
    return edges

def load_json_data(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data

def preprocess_iot23_data(file_path):
    data = load_json_data(file_path)

    # Extract the relevant data section from the JSON
    fields = []
    for record in data.get('recordSet', []):
        for field in record.get('field', []):
            column = field.get('source', {}).get('extract', {}).get('column')
            if column:
                fields.append(column)

    # Convert the JSON data to a DataFrame
    df = pd.DataFrame(data['data'])

    # Check if necessary columns exist and rename them for convenience
    if 'id.orig_h' in df.columns:
        df.rename(columns={'id.orig_h': 'srcid', 'id.resp_h': 'dstid', 'label': 'label'}, inplace=True)
    else:
        raise KeyError("Expected columns 'id.orig_h', 'id.resp_h', or 'label' not found in the dataset.")

    # Inspect the DataFrame to ensure it has the correct columns
    print("Columns in the DataFrame:", df.columns)

    # Create edge list
    edges = df[['srcid', 'dstid']].values.tolist()

    # Encode labels
    label_encoder = LabelEncoder()
    df['label'] = label_encoder.fit_transform(df['label'])

    return df, edges

def load_graph_data(training_config, device):
    dataset_name = training_config['dataset_name'].lower()
    should_visualize = training_config['should_visualize']

    if dataset_name == DatasetType.IOT23.name.lower():
        json_file = os.path.join(DATA_DIR_PATH, 'iot23.json')  # Change this to your actual file name

        if not os.path.isfile(json_file):
            raise FileNotFoundError(f"File not found: {json_file}")

        # Preprocess data
        df, edges = preprocess_iot23_data(json_file)

        # Build edge index
        src_col, dst_col = 'srcid', 'dstid'
        edges = pd.DataFrame(edges, columns=['source', 'target'])
        unique_ips = pd.concat([df[src_col], df[dst_col]]).unique()
        ip_encoder = LabelEncoder().fit(unique_ips)
        node_features = torch.eye(len(unique_ips))[ip_encoder.transform(unique_ips)]

        edge_index = torch.tensor([edges['source'].values, edges['target'].values], dtype=torch.long)

        # Labels
        labels = torch.tensor(df['label'].values, dtype=torch.long)

        # Convert to dense PyTorch tensors
        node_features = torch.tensor(node_features, dtype=torch.float, device=device)

        # Indices for train/val/test splits (example - modify as needed)
        num_nodes = len(df)
        train_indices = torch.arange(0, int(0.6 * num_nodes), dtype=torch.long, device=device)
        val_indices = torch.arange(int(0.6 * num_nodes), int(0.8 * num_nodes), dtype=torch.long, device=device)
        test_indices = torch.arange(int(0.8 * num_nodes), num_nodes, dtype=torch.long, device=device)

        return node_features, labels, edge_index, train_indices, val_indices, test_indices
    else:
        raise Exception(f'{dataset_name} not yet supported.')

# Example training config
training_config = {
    'dataset_name': 'IOT23',
    'should_visualize': True
}

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Load data (example usage)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
node_features, labels, edge_index, train_indices, val_indices, test_indices = load_graph_data(training_config, device)

# Visualize graph (if enabled)
if training_config['should_visualize']:
    def to_networkx(edge_index, labels):
        G = nx.Graph()
        G.add_nodes_from(range(len(labels)))
        edges = edge_index.t().tolist()
        G.add_edges_from(edges)
        return G

    G = to_networkx(edge_index, labels)
    plt.figure(figsize=(8, 6))
    nx.draw(G, with_labels=True, node_color=labels.numpy(), node_size=700, edge_color='gray', font_weight='bold')
    plt.title('Network Graph Visualization')
    plt.show()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


KeyError: 'data'

# GAT

In [5]:
# !pip install torch-scatter -f https://data.pyg.org/whl/torch-1.9.0+cu111.html
# !pip install torch-sparse -f https://data.pyg.org/whl/torch-1.9.0+cu111.html
# !pip install torch-cluster -f https://data.pyg.org/whl/torch-1.9.0+cu111.html
# !pip install torch-spline-conv -f https://data.pyg.org/whl/torch-1.9.0+cu111.html
!pip install torch-geometric

  Using cached torch_geometric-2.5.3-py3-none-any.whl (1.1 MB)


**torch.nn.functional**: Contains functions for different neural network operations, like dropout and activation functions.

t**orch_geometric.nn**: Contains neural network layers for graph-based computations, including GATConv.

In [6]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GATConv

Define the GAT Class:
**num_features**: Number of features in the input data.

**hidden_channels**: Number of hidden units in the first GAT layer.

**num_classes**: Number of output classes (e.g., attack/no attack).

**GATConv Layers**: The first layer (conv1) has 8 attention heads, and the second layer (conv2) has 1 attention head. Each attention head computes a weighted sum of the features of its neighbors.




In [7]:
class GAT(torch.nn.Module):
    def __init__(self, num_features, hidden_channels, num_classes):
        super(GAT, self).__init__()
        self.conv1 = GATConv(num_features, hidden_channels, heads=8, dropout=0.6)
        self.conv2 = GATConv(hidden_channels * 8, num_classes, heads=1, concat=True, dropout=0.6)


**forward Method**: Defines the forward pass of the GAT model.

*   Dropout: Randomly sets some elements of the input tensor to zero with a probability of 0.6, helping to prevent overfitting.
*   First GAT Layer: Applies the first GAT convolution layer.ELU
Activation: Applies the Exponential Linear Unit

*   List item
*   List item

(ELU) activation function to introduce non-linearity.
Second Dropout: Applies dropout again to the output of the first layer.
Second GAT Layer: Applies the second GAT convolution layer.
Log Softmax: Applies the log softmax function to the output, making it suitable for classification tasks by outputting log-probabilities.

In [8]:
def forward(self, x, edge_index):
    x = F.dropout(x, p=0.6, training=self.training)
    x = self.conv1(x, edge_index)
    x = F.elu(x)
    x = F.dropout(x, p=0.6, training=self.training)
    x = self.conv2(x, edge_index)
    return F.log_softmax(x, dim=1)


**Example Usage**

The above code is a binary classification task, such as identifying whether a node in a network is part of an attack or not. It includes two GAT convolution layers with dropout and ELU activation, making it capable of learning complex patterns in graph-structured data.

In [9]:
num_features = 16  # Example number of features
hidden_channels = 8
num_classes = 2  # For binary classification (attack/no attack)

model = GAT(num_features, hidden_channels, num_classes)
print(model)


GAT(
  (conv1): GATConv(16, 8, heads=8)
  (conv2): GATConv(64, 2, heads=1)
)


# Prepare environment

Create and Run a Quantum Circuit

In [ ]:
# Import necessary Qiskit components
from qiskit import QuantumCircuit, transpile, assemble
from qiskit.visualization import plot_histogram

# Create a Quantum Circuit with 1 qubit and 1 classical bit
qc = QuantumCircuit(1, 1)

# Apply a Hadamard gate to the qubit
qc.h(0)

# Measure the qubit
qc.measure(0, 0)

# Visualize the circuit
qc.draw('mpl')

# Use Aer's qasm_simulator
simulator = Aer.get_backend('qasm_simulator')

# Transpile the circuit for the simulator
compiled_circuit = transpile(qc, simulator)

# Assemble the circuit into a Qobj
qobj = assemble(compiled_circuit)

# Run the simulation and get the result
result = simulator.run(qobj).result()

# Get the counts (number of times each result was measured)
counts = result.get_counts(qc)

# Plot the results as a histogram
plot_histogram(counts)

MissingOptionalLibraryError: "The 'pylatexenc' library is required to use 'MatplotlibDrawer'. You can install it with 'pip install pylatexenc'."

Install Pytorch

In [ ]:

!pip install torch
import torch
import torch.nn as nn
import torch.optim as optim
# Define a simple neural network
class Net(nn.Module):
  def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)

  def forward(self, x):
        x = x.view(-1, 28 * 28)  # Flatten the input
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Set device to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize the network and move it to the device
net = Net().to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)

# Define a simple transformation for the MNIST dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

# Load the MNIST dataset
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Training loop
for epoch in range(2):
    # Train for 2 epochs
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = net(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f'Epoch {epoch + 1}, Loss: {running_loss / len(train_loader)}')

print('Finished Training')

# Save the trained model
torch.save(net.state_dict(), 'simple_net.pth')

NameError: name 'transforms' is not defined

I

```
# This is formatted as code
```

ntegrate a quantum layer using PennyLane with a classical deep learning model

In [ ]:
import pennylane as qml
from pennylane import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Define a quantum device
n_qubits = 4
dev = qml.device("default.qubit", wires=n_qubits)

# Define a quantum layer
@qml.qnode(dev)
def quantum_layer(inputs):
    for i in range(n_qubits):
        qml.RX(inputs[i], wires=i)
    for i in range(n_qubits):
        qml.CNOT(wires=[i, (i+1) % n_qubits])
    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]

# Integrate quantum layer with Keras
class QuantumLayer(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super(QuantumLayer, self).__init__(**kwargs)

    def call(self, inputs):
        return tf.convert_to_tensor([quantum_layer(inputs)], dtype=tf.float32)

# Build the model
model = Sequential()
model.add(Dense(16, activation='relu'))
model.add(QuantumLayer())
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model (using a sample dataset)
# model.fit(X_train, y_train, epochs=10, batch_size=32)

# Evaluate the model
# model.evaluate(X_test, y_test)


Selecting Feature

In [ ]:
from qiskit import Aer
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit_optimization.translators import from_qubo
from qiskit.algorithms import QAOA
from qiskit.utils import algorithm_globals, QuantumInstance
from qiskit.providers.aer import AerSimulator

# Define the number of features
num_features = 10

# Initialize a Quadratic Program, Specifies the number of features to select.
qp = QuadraticProgram()

# Add binary variables representing feature selection
for i in range(num_features):
    qp.binary_var(name=f'x{i}')

# Example weights and interactions (you need to define these based on your data)
weights = [0.5, 0.6, 0.4, 0.7, 0.3, 0.8, 0.2, 0.9, 0.1, 0.5]
interactions = {(i, j): 0.01 for i in range(num_features) for j in range(i+1, num_features)}

# Objective function: Maximize sum of selected feature weights and interactions
objective = sum(weights[i] * qp.binary_var(f'x{i}') for i in range(num_features))
objective += sum(interactions[(i, j)] * qp.binary_var(f'x{i}') * qp.binary_var(f'x{j}') for i, j in interactions)

# Add objective to the Quadratic Program
qp.maximize(linear=[weights[i] for i in range(num_features)], quadratic=interactions)

# If you want to constrain the number of features selected, add a constraint
k = 5  # Number of features to select
qp.linear_constraint(linear={f'x{i}': 1 for i in range(num_features)}, sense='==', rhs=k)

# Convert the Quadratic Program to a QUBO
qubo = qp.to_qubo()

# Setup QAOA
backend = AerSimulator()
quantum_instance = QuantumInstance(backend, seed_simulator=algorithm_globals.random_seed, seed_transpiler=algorithm_globals.random_seed)
qaoa = QAOA(quantum_instance=quantum_instance)
optimizer = MinimumEigenOptimizer(qaoa)

# Solve the QUBO problem using QAOA
result = optimizer.solve(qp)

# Extract selected features
selected_features = [i for i, x in enumerate(result.x) if x == 1]

print("Selected Features:", selected_features)


ModuleNotFoundError: No module named 'qiskit'

# Data Collection and Preprocessing

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder

# Load dataset from csv file
data = pd.read_csv('rootkit_malware_dataset.csv')

# Remove duplicates
data.drop_duplicates(inplace=True)

# Handle missing values
data.fillna(method='ffill', inplace=True)

#extract from the three columns using CountVectorizer to convert text to numerical
# Extract system calls
vectorizer = CountVectorizer()
system_call_features = vectorizer.fit_transform(data['system_calls']).toarray()
system_call_feature_names = [f"system_call_{i}" for i in range(system_call_features.shape[1])]
system_call_df = pd.DataFrame(system_call_features, columns=system_call_feature_names)

# Extract file changes
vectorizer = CountVectorizer()
file_change_features = vectorizer.fit_transform(data['file_changes']).toarray()
file_change_feature_names = [f"file_change_{i}" for i in range(file_change_features.shape[1])]
file_change_df = pd.DataFrame(file_change_features, columns=file_change_feature_names)

# Extract network traffic patterns
vectorizer = CountVectorizer()
network_traffic_features = vectorizer.fit_transform(data['network_traffic']).toarray()
network_traffic_feature_names = [f"network_traffic_{i}" for i in range(network_traffic_features.shape[1])]
network_traffic_df = pd.DataFrame(network_traffic_features, columns=network_traffic_feature_names)

# each unique token is a feature
# Combine extracted features with the original dataset
data = pd.concat([data, system_call_df, file_change_df, network_traffic_df], axis=1)

# Drop the original raw columns as they are now encoded
data.drop(columns=['system_calls', 'file_changes', 'network_traffic'], inplace=True)

# normalize and Feature Scaling
# will make them have a mean of 0 and stadndard deviation of 1
scaler = StandardScaler()
numerical_features = ['cpu_usage', 'memory_usage', 'network_activity']
data[numerical_features] = scaler.fit_transform(data[numerical_features])

# One-hot Encoding for categorical variables, convert variables into binary
categorical_features = ['protocol', 'device_type']
encoder = OneHotEncoder()
encoded_features = encoder.fit_transform(data[categorical_features]).toarray()

# Combine encoded features with the dataset
encoded_feature_names = encoder.get_feature_names_out(categorical_features)
encoded_df = pd.DataFrame(encoded_features, columns=encoded_feature_names)
data = pd.concat([data, encoded_df], axis=1)
data.drop(columns=categorical_features, inplace=True)

# Split data into training, validation, and testing sets
# training = 70%, validation = 15%(evaluate by tuning parameters after training on the training data), testing = 15%
train_data, temp_data = train_test_split(data, test_size=0.3, random_state=42)
val_data, test_data = train_test_split(temp_data, test_size=0.5, random_state=42)

# Save processed datasets without including index column
train_data.to_csv('train_data.csv', index=False)
val_data.to_csv('val_data.csv', index=False)
test_data.to_csv('test_data.csv', index=False)


FileNotFoundError: [Errno 2] No such file or directory: 'rootkit_malware_dataset.csv'

# Optimization (QAOA) and tuning weight for penalty



In [ ]:
import numpy as np
from qiskit import Aer, execute
from qiskit.circuit.library import QAOAAnsatz
from qiskit.optimization import QuadraticProgram
from qiskit.optimization.algorithms import MinimumEigenOptimizer
from qiskit.aqua.algorithms import QAOA
from qiskit.aqua import QuantumInstance
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import accuracy_score

# Example data and feature importance (dummy data)
n_features = 10
feature_importance = np.random.rand(n_features)
accuracy_weights = np.random.rand(n_features)  # Weights based on feature importance for accuracy

# Define the Quadratic Program
def define_quadratic_program(alpha, beta):
    qp = QuadraticProgram()
    for i in range(n_features):
        qp.binary_var(name=f'x{i}')
    objective = alpha * accuracy_weights - beta
    qp.maximize(linear=objective)
    return qp

# QAOA ansatz and quantum instance
qaoa_ansatz = QAOAAnsatz(optimizer='COBYLA')
backend = Aer.get_backend('qasm_simulator')
quantum_instance = QuantumInstance(backend, shots=1024)

# Solve using QAOA
def solve_qaoa(qp):
    qaoa = QAOA(qaoa_ansatz, quantum_instance)
    optimizer = MinimumEigenOptimizer(qaoa)
    result = optimizer.solve(qp)
    selected_features = [i for i, x in enumerate(result.x) if x > 0.5]
    return selected_features

# Evaluate the model
def evaluate_model(selected_features, X, y):
    X_selected = X[:, selected_features]
    X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.2, random_state=42)
    # Train a simple classifier (e.g., logistic regression) and evaluate accuracy
    from sklearn.linear_model import LogisticRegression
    clf = LogisticRegression()
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy

# Tuning function
def tune_weights(X, y, alpha_range, beta_range, cv_folds=3):
    best_alpha = None
    best_beta = None
    best_score = -np.inf

    for alpha in alpha_range:
        for beta in beta_range:
            qp = define_quadratic_program(alpha, beta)
            kf = KFold(n_splits=cv_folds)
            avg_accuracy = 0
            avg_features = 0

            for train_index, test_index in kf.split(X):
                X_train, X_val = X[train_index], X[test_index]
                y_train, y_val = y[train_index], y[test_index]

                selected_features = solve_qaoa(qp)
                accuracy = evaluate_model(selected_features, X_train, y_train)
                avg_accuracy += accuracy / cv_folds
                avg_features += len(selected_features) / cv_folds

            composite_score = alpha * avg_accuracy - beta * avg_features
            if composite_score > best_score:
                best_score = composite_score
                best_alpha = alpha
                best_beta = beta

    return best_alpha, best_beta

# Example usage
X = np.random.rand(100, n_features)  # Dummy feature data
y = np.random.randint(0, 2, 100)  # Dummy target data
alpha_range = np.linspace(0.1, 1.0, 10)
beta_range = np.linspace(0.01, 0.1, 10)

best_alpha, best_beta = tune_weights(X, y, alpha_range, beta_range)
print("Best alpha:", best_alpha)
print("Best beta:", best_beta)


TO COMPARE COMPOSITE SCORE BEFORE AND AFTER OPTIMIZATION.

HIGHER COMPOSITE = BETTER PERFORMANCE

In [ ]:

# Define a baseline alpha and beta
baseline_alpha = 0.5
baseline_beta = 0.05

# Calculate the initial composite score
def initial_composite_score(X, y, alpha, beta, cv_folds=3):
    qp = define_quadratic_program(alpha, beta)
    kf = KFold(n_splits=cv_folds)
    avg_accuracy = 0
    avg_features = 0

    for train_index, test_index in kf.split(X):
        X_train, X_val = X[train_index], X[test_index]
        y_train, y_val = y[train_index], y[test_index]

        selected_features = solve_qaoa(qp)
        accuracy = evaluate_model(selected_features, X_train, y_train)
        avg_accuracy += accuracy / cv_folds
        avg_features += len(selected_features) / cv_folds

    composite_score = alpha * avg_accuracy - beta * avg_features
    return composite_score

initial_score = initial_composite_score(X, y, baseline_alpha, baseline_beta)
print("Initial composite score:", initial_score)

# Perform tuning to find the best alpha and beta
best_alpha, best_beta = tune_weights(X, y, alpha_range, beta_range)
print("Best alpha:", best_alpha)
print("Best beta:", best_beta)

# Calculate the post-optimization composite score
optimized_score = initial_composite_score(X, y, best_alpha, best_beta)
print("Optimized composite score:", optimized_score)

# Compare scores
improvement = optimized_score - initial_score
print("Improvement in composite score:", improvement)


#  Train ML model

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd

# Load and preprocess your dataset (assuming data is preprocessed and split into X_train, X_test, y_train, y_test)
# Example:
# X_train = pd.read_csv('X_train.csv')
# X_test = pd.read_csv('X_test.csv')
# y_train = pd.read_csv('y_train.csv')
# y_test = pd.read_csv('y_test.csv')

# Select features
X_train_selected = X_train[:, selected_features]
X_test_selected = X_test[:, selected_features]

# Train model
model = RandomForestClassifier()
model.fit(X_train_selected, y_train)

# Make predictions
predictions = model.predict(X_test_selected)

# Evaluate model
accuracy = accuracy_score(y_test, predictions)
precision = precision_score(y_test, predictions)
recall = recall_score(y_test, predictions)
f1 = f1_score(y_test, predictions)

print(f"Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, F1-Score: {f1}")
